# Exploration des résultats de l'analyse RNA-seq

Nettoyage des données résiduelles :

In [1]:
rm -f GUP-1_6p_on_chr6p-cox.sam GUP-1_6p_on_chr6p-cox.sam.gz

## Accès aux données

Les résultats de l'analyse RNA-seq se trouvent dans le répertoire `/srv/data/meg-m1-ue5/fil_rouge_unix`.

Vous n'avez pas le droit d'écrire dans ce répertoire, mais vous pouvez le parcourir et ouvrir ses fichiers.

**Question :** Affichez le contenu du répertoire `/srv/data/meg-m1-ue5/fil_rouge_unix` avec les détails :

In [2]:
ls -l /srv/data/meg-m1-ue5/fil_rouge_unix

total 36
drwxr-xr-x 2 ppoulain 1013 4096 Oct 14 11:35 chr6p_annotations
drwxr-xr-x 2 ppoulain 1013 4096 Oct 14 11:35 chr6p_genomes
drwxr-xr-x 2 ppoulain 1013 4096 Oct 14 11:35 chr6p_index
drwxr-xr-x 2 ppoulain 1013 4096 Oct 14 11:35 chr6p_map
drwxr-xr-x 2 ppoulain 1013 4096 Oct 14 11:35 chr6p_samples
drwxr-xr-x 2 ppoulain 1013 4096 Oct 14 11:35 chr6p_trim
-rw-r--r-- 1 ppoulain 1013 4031 Oct 14 11:35 workflow_20201014-112523.log
-rwxr-xr-x 1 ppoulain 1013 5994 Oct 14 11:35 workflow.sh


**Question :** Affichez l'arboresence (les répertoires, les sous-répertoires et les fichiers) de ce répertoire.

In [3]:
tree /srv/data/meg-m1-ue5/fil_rouge_unix

/srv/data/meg-m1-ue5/fil_rouge_unix
├── chr6p_annotations
│   ├── chr6-cox.gtf
│   └── chr6-pgf.gtf
├── chr6p_genomes
│   ├── chr6p-cox.fa
│   └── chr6p-pgf.fa
├── chr6p_index
│   ├── chr6p-cox.1.ht2
│   ├── chr6p-cox.2.ht2
│   ├── chr6p-cox.3.ht2
│   ├── chr6p-cox.4.ht2
│   ├── chr6p-cox.5.ht2
│   ├── chr6p-cox.6.ht2
│   ├── chr6p-cox.7.ht2
│   ├── chr6p-cox.8.ht2
│   ├── chr6p-cox.fa
│   ├── chr6p-pgf.1.ht2
│   ├── chr6p-pgf.2.ht2
│   ├── chr6p-pgf.3.ht2
│   ├── chr6p-pgf.4.ht2
│   ├── chr6p-pgf.5.ht2
│   ├── chr6p-pgf.6.ht2
│   ├── chr6p-pgf.7.ht2
│   ├── chr6p-pgf.8.ht2
│   └── chr6p-pgf.fa
├── chr6p_map
│   ├── GUP-1_6p_on_chr6p-cox.bai
│   ├── GUP-1_6p_on_chr6p-cox.bam
│   ├── GUP-1_6p_on_chr6p-cox.log
│   ├── GUP-1_6p_on_chr6p-pgf.bai
│   ├── GUP-1_6p_on_chr6p-pgf.bam
│   ├── GUP-1_6p_on_chr6p-pgf.log
│   ├── GUP-3_6p_on_chr6p-cox.bai
│   ├── GUP-3_6p_on_chr6p-cox.bam
│   ├── GUP-3_6p_on_chr6p-cox.log
│   ├── GUP-3_6p_on_chr6p-pgf.bai
│   ├── GUP-3_6p_on_chr6p-pgf.bam
│   └── GU

**Question :** Quel espace disque est occupé par l'ensemble des données dans ce répertoire ?

In [4]:
du -sh /srv/data/meg-m1-ue5/fil_rouge_unix/

961M	/srv/data/meg-m1-ue5/fil_rouge_unix/


**Question :** Affichez la taille des fichiers `.bam` issus du mapping.

In [5]:
ls -lh /srv/data/meg-m1-ue5/fil_rouge_unix/chr6p_map/*.bam

-rw-r--r-- 1 ppoulain 1013 74M Oct 14 11:35 /srv/data/meg-m1-ue5/fil_rouge_unix/chr6p_map/GUP-1_6p_on_chr6p-cox.bam
-rw-r--r-- 1 ppoulain 1013 74M Oct 14 11:35 /srv/data/meg-m1-ue5/fil_rouge_unix/chr6p_map/GUP-1_6p_on_chr6p-pgf.bam
-rw-r--r-- 1 ppoulain 1013 55M Oct 14 11:35 /srv/data/meg-m1-ue5/fil_rouge_unix/chr6p_map/GUP-3_6p_on_chr6p-cox.bam
-rw-r--r-- 1 ppoulain 1013 56M Oct 14 11:35 /srv/data/meg-m1-ue5/fil_rouge_unix/chr6p_map/GUP-3_6p_on_chr6p-pgf.bam


## Bam et Sam

Les fichiers `.bam` sont la version binaire des fichiers `.sam`.

**Question :** `.bam` vs `.sam`, quels sont les avantages et les inconvénients ?

Inconvénient :
- On ne peut pas les ouvrir avec un éditeur de texte ou les manipuler avec les outils Unix classiques (`cat`, `grep`, `head`, `awk`...)

Avantages :
- Ils prennent peu de place sur le disque.
- La plupart des outils de bioinfo orientés NGS peuvent manipuler ce type de fichier.

## Manipulation avec samtools

### Retour sur le workflow

Le logiciel `samtools` est utilisé plusieurs fois dans le script `workflow.sh`.

**Question :** Identifiez à quelles étapes `samtools` intervient et pour quoi faire exactement.

### étape 3 « map samples on genome + index mapping »
Ligne de commande :
```
hisat2 -x ../${index_dir}/${genome_name} \
            --rna-strandness FR \
            -1 ../${trim_dir}/${sample_name}_trim_1P.fastq.gz \
            -2 ../${trim_dir}/${sample_name}_trim_2P.fastq.gz \
            2> ${sample_name}_on_${genome_name}.log | \
            samtools sort -T ${sample_name}_on_${genome_name} -o ${sample_name}_on_${genome_name}.bam -
```

Le logiciel d'alignement `hisat2` renvoie l'alignement au format sam. On récupère cet alignement à la volée avec le pipe (`|`) à la fin de la ligne (`2> ${sample_name}_on_${genome_name}.log | \`) puis on le redirige vers `samtools` pour trier les reads et produire un fichier binaire `.bam`.


### étape 3 « map samples on genome + index mapping » (encore)
Ligne de commande :
```
samtools index -b ${sample_name}_on_${genome_name}.bam
```

`samtools` est utilisé ici pour indexé le fichier `.bam` produit juste avant. Le fichie index ainsi créé porte l'extension `.bai`.


### étape 4 « count number of mapped reads (with and without mismatches) »
Ligne de commande :
```
total_reads=$(samtools view ${sample_name}_on_${genome_name}.bam ${genome_name}:28734408-33383765 | uniq | wc -l)
nomismatch_reads=$(samtools view ${sample_name}_on_${genome_name}.bam ${genome_name}:28734408-33383765 | grep "NM:i:0" | uniq | wc -l)
```

On utilise `samtools` pour fouiller dans les résultats d'alignement pour une région chromosomique particulière (`28734408-33383765`).

La commande `samtools view` permet de transformer une fichier `.bam` en `.sam`.

## Samtools view

La commande `samtools view` permet de transformer une fichier `.bam` en `.sam` et de visualiser certaines propriétés du fichier `.bam`.

**Question :** Dans l'aide de `samtools view` ([ici](http://www.htslib.org/doc/samtools-view.html)), trouvez quelle option permet d'afficher **uniquement** les métadonnées (*header*) d'un fichier `.bam`.

Ici, c'est l'option `-H` qui nous intéresse :
> `-H        Output the header only.`

**Question :** Affichez les métadonnées du fichier `GUP-1_6p_on_chr6p-cox.bam`. Pensez à bien indiquer le chemin complet du fichier (`/srv/data/chemin/à/déterminer/GUP-1_6p_on_chr6p-cox.bam`).

In [6]:
samtools view -H /srv/data/meg-m1-ue5/fil_rouge_unix/chr6p_map/GUP-1_6p_on_chr6p-cox.bam

@HD	VN:1.0	SO:coordinate
@SQ	SN:chr6p-cox	LN:60000000
@PG	ID:hisat2	PN:hisat2	VN:2.2.1	CL:"/srv/miniconda3/envs/rnaseq-meg-m1-ue5/bin/hisat2-align-s --wrapper basic-0 -x ../chr6p_index/chr6p-cox --rna-strandness FR --read-lengths 100,75,68,77,71,70,65,64,74,69,67,72,66,82,78,73,60,53,98,76,49,51,96,61,58,55,52,97,90,95,62,59,63,57,39,56,94,93,48,47,88,54,92,99,50,38,79,84,86,41,37,85,83,46,42,40,91,87,45,89,81,43,80,44,36 -1 /tmp/95785.inpipe1 -2 /tmp/95785.inpipe2"
@PG	ID:samtools	PN:samtools	PP:hisat2	VN:1.11	CL:samtools sort -T GUP-1_6p_on_chr6p-cox -o GUP-1_6p_on_chr6p-cox.bam -
@PG	ID:samtools.1	PN:samtools	PP:samtools	VN:1.10	CL:samtools view -H /srv/data/meg-m1-ue5/fil_rouge_unix/chr6p_map/GUP-1_6p_on_chr6p-cox.bam


**Question :** Retrouvez dans ces métadonnées le logiciel d'alignement utilisé.

Lignes 3 et 4, on retrouve la mention de `hisat2` avec même la ligne de commande complète.

### Comparaison de taille de fichiers

Convertissez le fichier `GUP-1_6p_on_chr6p-cox.bam` en fichier `.sam` dans votre répertoire courant avec la commande suivante :
```
samtools view /srv/data/chemin/à/déterminer/GUP-1_6p_on_chr6p-cox.bam > GUP-1_6p_on_chr6p-cox.sam
```

Attention, n'oubliez pas la redirection `>`. Sinon vous risquez de faire planter votre notebook Jupyter.

In [7]:
samtools view /srv/data/meg-m1-ue5/fil_rouge_unix/chr6p_map/GUP-1_6p_on_chr6p-cox.bam > GUP-1_6p_on_chr6p-cox.sam

**Question :** Quelle est la taille du fichier `/srv/data/chemin/à/déterminer/GUP-1_6p_on_chr6p-cox.bam` ? 

In [8]:
ls -lh /srv/data/meg-m1-ue5/fil_rouge_unix/chr6p_map/GUP-1_6p_on_chr6p-cox.bam

-rw-r--r-- 1 ppoulain 1013 74M Oct 14 11:35 /srv/data/meg-m1-ue5/fil_rouge_unix/chr6p_map/GUP-1_6p_on_chr6p-cox.bam


**Question :** Quelle est la taille du fichier `GUP-1_6p_on_chr6p-cox.sam` obtenu ? 

In [9]:
ls -lh GUP-1_6p_on_chr6p-cox.sam

-rw-rw-r-- 1 ppoulain ppoulain 387M Oct 14 19:53 GUP-1_6p_on_chr6p-cox.sam


**Question :** Quel est le rapport de taille entre les deux fichiers ?

$$
\frac{387}{75} \approx 5.2
$$

Compressez maintenant le fichier `.sam` avec gzip :
```
gzip -k GUP-1_6p_on_chr6p-cox.sam
```

In [10]:
gzip -k GUP-1_6p_on_chr6p-cox.sam

**Question :** Quelle est l'utilité de l'option `-k` de la commande `gzip` ?

L'option `-k` permer de conserver le fichier non compressé. Habituellement, il est supprimé.

**Question :** Quelle est la taille du fichier `GUP-1_6p_on_chr6p-cox.sam.gz` ?

In [11]:
ls -lh GUP-1_6p_on_chr6p-cox.sam.gz

-rw-rw-r-- 1 ppoulain ppoulain 69M Oct 14 19:53 GUP-1_6p_on_chr6p-cox.sam.gz


**Question :** La taille du fichier `.sam.gz` est plus petite que celle du fichier `.bam` initial.
Pourquoi dans ce cas, utilisez quand le format `.bam` ?

Un fichier au format `.bam` peut être indexé pour ensuite être parcouru très rapidement. Ce n'est pas possible avec un fichier compressé `.sam.gz`.

### Recherche d'un read pairé

**Question :** Affichez les 10 premières lignes du fichier `.sam`.

In [12]:
head GUP-1_6p_on_chr6p-cox.sam

HWI-ST365:251:D0RP0ACXX:2:1109:7427:33385	177	chr6p-cox	61313	60	5S95M	=	61353	100	GGAAAGAGAGTCAGATGATAAGAGGGTCAAAATTATGTTTATCTTAGGAAAAGTAGAATAGAAAATTTATAAGCAGATTAAAAACACATAATAAAAGTAG	>:CDBFDFFHCAHFHEHHGHGGEGIIIGJIGHHGHGGEEHHCHDIIJHGGJIIIHHHIHGEIHFGEF<IGHEIIJIGJJIHGE?IIJHGHHHFFFFFCBB	AS:i:-5	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:95	YS:i:0	YT:Z:DP	XS:A:+	NH:i:1
HWI-ST365:251:D0RP0ACXX:2:1109:7427:33385	113	chr6p-cox	61353	60	55M	=	61313	-100	AGGAAAAGTAGAATAGAAAATTTATAAGCAGATTAAAAACACATAATAAAAGTAG	:0*HFBDD<C?9**GDFFEBE>CC@>F4:4@EB;CF@<2A?FF>DAA?;BA41::	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:55	YS:i:-5	YT:Z:DP	XS:A:-	NH:i:1
HWI-ST365:251:D0RP0ACXX:2:2115:4373:61541	163	chr6p-cox	62660	60	98M	=	62759	145	CGTACCTTAGTTCAACCATTGTGGAAGACAGTGTGGTGTTTCCTCAAGGATCTAAAACTAGAAATACCATTTGACCCAGCAATCCCATTACTGGGTAT	B+14B+AB<FC?FEDHB=<CHF4C<?C;F;BEDDFE1??BF9DBBAF@FHFADGACHDH8=C7@DE@@=;CCEC:=A=ABD9@;>ACECA@CDC?;5@	AS:i:-8	ZS:i:-11	XN:i:0	XM:i:2	XO:i:0	XG:i:0	NM:i:2	MD:Z:5A40T51	YS:i:-2	YT:Z:CP	X

**Question :** Vérifiez que le read `HWI-ST365:251:D0RP0ACXX:2:2115:4373:61541` est bien présent dans sa version pairée (R1 et R2). Pour cela recherche le motif `HWI-ST365:251:D0RP0ACXX:2:2115:4373:61541` dans le fichier `.sam` (ou directement depuis le fichier `.bam` avec `samtools view`).

In [13]:
grep HWI-ST365:251:D0RP0ACXX:2:2115:4373:61541 GUP-1_6p_on_chr6p-cox.sam

HWI-ST365:251:D0RP0ACXX:2:2115:4373:61541	163	chr6p-cox	62660	60	98M	=	62759	145	CGTACCTTAGTTCAACCATTGTGGAAGACAGTGTGGTGTTTCCTCAAGGATCTAAAACTAGAAATACCATTTGACCCAGCAATCCCATTACTGGGTAT	B+14B+AB<FC?FEDHB=<CHF4C<?C;F;BEDDFE1??BF9DBBAF@FHFADGACHDH8=C7@DE@@=;CCEC:=A=ABD9@;>ACECA@CDC?;5@	AS:i:-8	ZS:i:-11	XN:i:0	XM:i:2	XO:i:0	XG:i:0	NM:i:2	MD:Z:5A40T51	YS:i:-2	YT:Z:CP	XS:A:-	NH:i:1
HWI-ST365:251:D0RP0ACXX:2:2115:4373:61541	83	chr6p-cox	62759	60	46M	=	62660	-145	TTCCCAAACGATTGTAAGTCATTCTACTATAAAGACACATGCACAG	0)EGFD=HBAEF@EEC:34@C>HFBIEGHEFFHFF?AHDDFFD@@?	AS:i:-2	ZS:i:-2	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:1A44	YS:i:-8	YT:Z:CP	XS:A:-	NH:i:1


In [14]:
samtools view /srv/data/meg-m1-ue5/fil_rouge_unix/chr6p_map/GUP-1_6p_on_chr6p-cox.bam | grep HWI-ST365:251:D0RP0ACXX:2:2115:4373:61541

HWI-ST365:251:D0RP0ACXX:2:2115:4373:61541	163	chr6p-cox	62660	60	98M	=	62759	145	CGTACCTTAGTTCAACCATTGTGGAAGACAGTGTGGTGTTTCCTCAAGGATCTAAAACTAGAAATACCATTTGACCCAGCAATCCCATTACTGGGTAT	B+14B+AB<FC?FEDHB=<CHF4C<?C;F;BEDDFE1??BF9DBBAF@FHFADGACHDH8=C7@DE@@=;CCEC:=A=ABD9@;>ACECA@CDC?;5@	AS:i:-8	ZS:i:-11	XN:i:0	XM:i:2	XO:i:0	XG:i:0	NM:i:2	MD:Z:5A40T51	YS:i:-2	YT:Z:CP	XS:A:-	NH:i:1
HWI-ST365:251:D0RP0ACXX:2:2115:4373:61541	83	chr6p-cox	62759	60	46M	=	62660	-145	TTCCCAAACGATTGTAAGTCATTCTACTATAAAGACACATGCACAG	0)EGFD=HBAEF@EEC:34@C>HFBIEGHEFFHFF?AHDDFFD@@?	AS:i:-2	ZS:i:-2	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:1A44	YS:i:-8	YT:Z:CP	XS:A:-	NH:i:1


**Question :** À partir des *flags* (ici 163 et 83), utilisez l'outil [Decoding SAM flags](https://broadinstitute.github.io/picard/explain-flags.html) pour vérifier que les deux alignements que vous avez obtenus correspondent bien à des reads pairés.

En recherchant les *flags* 163 et 83, on obtient les informations suivantes :

*Flag* 163 :
- read paired (0x1)
- read mapped in proper pair (0x2)
- mate reverse strand (0x20)
- second in pair (0x80)

*Flag* 83 :
- read paired (0x1)
- read mapped in proper pair (0x2)
- read reverse strand (0x10)
- first in pair (0x40)

<div class="alert alert-block alert-info">
Pour en savoir encore plus sur l'analyse de données RNA-seq, n'oubliez pas de suivre l'option RNA-seq de Claire Vandiedonck et Sandrine Caburet en M2 !
</div>